In [2]:
import os
import sys
import numpy as np
import pandas as pd
from collections import defaultdict
from python_modules.utils.metrics import EvalSurvDCalib

test_annotation_file = '/Users/nja/Downloads/enrico_example/rats/master.csv'
user_annotation_folder = '/Users/nja/Downloads/enrico_example/rats'
test_splits_file = '/Users/nja/Downloads/enrico_example/rats/test_splits.csv'
n_splits = 25
ibs_grid_length = 100
phase_codename  = 'Trial Run'

def evaluate(test_annotation_file, user_annotation_folder, phase_codename, test_splits_file, n_splits, ibs_grid_length, **kwargs):
    # maybe user file can be a folder instead, cancer -> splits
    # test_splits - ideally should be a dict of split idx, currently it is a csv file where each col specifies the indices in a split
    # user_prediction_files = os.listdir(user_annotation_folder)
    test_splits = pd.read_csv(test_splits_file)

    d_calibration_statistic = np.zeros(n_splits)
    integrated_brier_score = np.zeros(n_splits)
    antolini_concordance = np.zeros(n_splits)

    master = pd.read_csv(test_annotation_file)
    status = master["status"].values
    time = master["time"].values

    output = defaultdict(list)
    cancer = 'Test'

    for i in range(n_splits):
       
        test_ix = test_splits.iloc[:, i].dropna().values
        
        predictions = pd.read_csv(os.path.join(user_annotation_folder,f'split_{i + 1}.csv')).T
        predictions.index = predictions.index.astype(int)
        eval_surv = EvalSurvDCalib(
            surv=predictions,
            # Splits were created from R so need to
            # switch from 1 to 0 indexing.
            durations=time[test_ix - 1],
            events=status[test_ix - 1],
            censor_surv="km",
            steps="post",
        )
        antolini_concordance[i] = eval_surv.concordance_td()
        print(antolini_concordance)
        d_calibration_statistic[i] = eval_surv.d_calibration_()
        print(d_calibration_statistic)
        integrated_brier_score[i] = eval_surv.integrated_brier_score(
            time_grid=np.linspace(
                np.min(time[test_ix - 1]),
                np.max(time[test_ix - 1]),
                ibs_grid_length,
            )
        )
        print(integrated_brier_score)

    cancer_dict = {cancer: {'antolini_concordance':np.median(antolini_concordance),'d_calibration_statistic':np.median(d_calibration_statistic),'integrated_brier_score':np.median(integrated_brier_score)}}
    
    output['result'].append(cancer_dict)
    
    return output 

ModuleNotFoundError: No module named 'python_modules'